In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
model = P3alphaRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2639 (20.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
import optuna as op
def objective(trial):
    # Optuna suggest params
    topK = trial.suggest_int("topK", 10, 1000)
    alpha = trial.suggest_uniform("alpha", 0.01, 1.0)
    normalize_similarity = trial.suggest_categorical("normalize_similarity", [True, False])
    
    recommender = model(URM_train)
    recommender.fit(**trial.params)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    "topK": 84,
    "alpha": 0.3904145688038595,
    "normalize_similarity": True,
}
#{"topK": 84, "alpha": 0.3904145688038595, "normalize_similarity": true}
study_name = "p3alpha-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

In [5]:
study.best_params

{'topK': 76, 'alpha': 0.377201600381895, 'normalize_similarity': True}

In [6]:
final = model(URM_train_validation)
final.fit(**study.best_params)

P3alphaRecommender: URM Detected 225 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 119 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4880.62 column/sec. Elapsed time 4.55 sec


In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

In [ ]:
## MAP = .046439